In [1]:
import renom as rm
import renom.graph as rmg
import torch
import numpy as np

In [3]:
torch.__version__

'1.4.0'

In [4]:
rm.__version__

'3.1.0b0'

## LAYER NORMALIZATION

Test on ReNom dev branch 31/03/2020

In [39]:
data = np.random.rand(3, 4)
data

array([[0.76572264, 0.96524121, 0.83518996, 0.86834323],
       [0.63784961, 0.27693641, 0.62184479, 0.40996314],
       [0.42499187, 0.86686037, 0.72939002, 0.92529927]])

In [40]:
torch_data = torch.FloatTensor(data)
torch_layer = torch.nn.LayerNorm(torch_data.shape[-1])

torch_layer(torch_data).detach().numpy()

array([[-1.2919891 ,  1.4827303 , -0.32590213,  0.1351632 ],
       [ 1.0022359 , -1.3900758 ,  0.8961478 , -0.5083087 ],
       [-1.6105705 ,  0.673002  , -0.03744362,  0.97501373]],
      dtype=float32)

In [41]:
rm.set_cuda_active(False)
rm_data = rmg.DynamicVariable(data)
renom_layer = rmg.LayerNormalize()

print(renom_layer(rm_data).numpy)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [42]:
rm.set_cuda_active(True)
rm_data = rmg.DynamicVariable(data)
renom_layer = rmg.LayerNormalize()

print(renom_layer(rm_data).numpy)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


## RESHAPE OPERATOR
Test on ReNom dev branch 31/03/2020

Can not change shape of the batch-size dimension

In [48]:
data = np.random.rand(3, 4, 5)
rm_data = rmg.DynamicVariable(data)

rm_data.reshape(3*4, 5)

AssertionError: First dimension of shape should be set to None

## GET ITEM OPERATOR
Test on ReNom carl/add-batchsize-getitem branch

(Only the getitem operator in this branch can slide data in the first dimension - batch dimension). Carl made this branch for me as a hotfix.

31/03/2020

In [46]:
#The getitem operator in dev branch will throw this error when trying to slide the first dimension 
a = rmg.DynamicVariable(data)
print(a.shape)
a[0]

(3, 4)


AssertionError: 

## PERFORMANCE

Test on ReNom carl/add-batchsize-getitem branch 31/03/2020

Bellow is a short example showing the performance issue in ReNom

In [27]:
rm.set_cuda_active(True)
a = np.random.rand(100, 50, 50)
b = np.random.rand(100, 50, 50)
a = rmg.DynamicVariable(a)
b = rmg.DynamicVariable(b)

In [28]:
import time
#A function for batch matrix multiplication
def batch_matmul(a, b): 
    result = list()
    t = time.time()
    for i in range(a.shape[0].value):
        mul = a[i]@b[i]
        # mul.shape == (50, 50)
        mul = mul.reshape(None, -1, 1) #Add new axis, because Reshape Operator 
        #does not work on the first dim so I need to do trick like this
        mul = mul.transpose(2, 0, 1)
        # mul.shape == (1, 50, 50)
        result.append(mul)
    print("Mul time: ", time.time()-t)
    t2 = time.time()
    concat = rmg.concatenate(result)
    print("Concat time: ", time.time()-t2)
    print("Total time: ", time.time()-t)
    return concat

In [29]:
# First, I just perform batch_matmul on a and b, pretty fast
c = batch_matmul(a, b)

Mul time:  0.31295084953308105
Concat time:  0.01199030876159668
Total time:  0.32543182373046875


In [24]:
#When I use the output of the first time as input for the second time, it runs so slow
d = batch_matmul(a, c)

Mul time:  8.440852165222168
Concat time:  0.3967585563659668
Total time:  8.838108777999878


In [26]:
#If I convert c to numpy, and then init a new DynamicVariable, it can run as fast as the first time
c = rmg.DynamicVariable(c.numpy)
batch_matmul(a, c)

Mul time:  0.24999189376831055
Concat time:  0.011896848678588867
Total time:  0.2620413303375244


In [30]:
#When I use the output of the second time as input for the third time, it even get slower
e = batch_matmul(a, d)

Mul time:  17.22134518623352
Concat time:  0.3893117904663086
Total time:  17.61120867729187


In [31]:
#And here is the 4th time LOL
f = batch_matmul(a, e)

Mul time:  1120.5647423267365
Concat time:  38.54055213928223
Total time:  1159.1054661273956


In [ ]:
#This is basicly the reason why my model runs so slow. The Getitem Operator runs slower and slower after times